# Milestone 2

Here's your guidance on milestone 2. The big picture is as follows: now that you understand how to make networks and measure centrality, let's shift into predicting centrality using machine learning.

The question is: given all the other centrality measures that I already have on a node, can I predict its centrality? E.g. given degree and betwenness centrality, can I predict closeness? Note that centralities are rankings. So you're only dealing with integers (rank 1, rank 2, rank 3...).
Here is the methodology to answer this question:
- for each type of network (random, SF, SM), create 100 instances equally split in networks of 100 nodes, 200 nodes, 400 nodes, and 800 nodes. (So you'd have 25 SF networks with 100 nodes for instance). In each instance, compute at least five different centrality measures for each node, and turn them into rankings. The result should go into a data.csv file, with headers such as

|networkType|networkSize|instanceNumber|nodeNumber|degree rank|closeness rank|betweenness rank|...|
|-----------|-----------|--------------|----------|-----------|--------------|-------------------|---|
|scale-free | 100| 1| 1| 6| 2| 20| ...|
|scale-free | 100| 1| 2| 5| 3| 21| ...|
|scale-free | 100| 1| 4| 10| 4| 19| ...|
    
    
(The above shows three nodes from one instance of a scale-free graph of size 100)

- compute the pairwise correlations between the rankings across all network types, as well as within each network type (random, SF, SM). We want this as a 'baseline'. The idea is that we should be able to make accurate predictions even where there didn't appear to be a linear correlation.
- we'll use machine learning algorithms for predictions. The logic is that we first try to make coarse predictions. If it doesn't work (=inaccurate) then we get even more coarse. If it works, we can try to be more precise. In other words, we don't start shooting for the moon but we try to find a middle ground (think of it as a binary search). Our starting point will be to predict whether one centrality of a node is in the top 25% based on the other centrality. For instance, given the ranks in degree, closeness, etc., is the node's betweenness in the top 25% or not (yes/no)? This is a binary classification. The steps would be to create the class outcome columns (data preparation). Then, for each prediction, ensure that the original centrality is removed (e.g. if you predict top 25% degree then you should not be using degree rank!), balance the data (so we have 50-50 of yes/no), and predict. I recommend using at least three different algorithms (e.g., decision trees, support vector machine, random forests; see textbook chapters on the last two) and a ten-fold cross-validation (also shown in the textbook). Balancing and cross-validation will be discussed in the upcoming Tuesday class. We'd like results (=accuracy) to be presented overall as well as divided per network type (so that we can see e.g. if we're better at making predictions in small-world than in random networks).

This milestone also comes with a bonus of 10% of your milestone 2 grade. That is, if you achieve the bonus in its entirety, whatever grade you got will be multiplied by 1.10. The bonus consists of doing all of the above on the forth type of network: small-world scale-free. Which means you need to be able to demonstrate that you can generate such networks for starters (fit of degree distribution / low average path length / high clustering). You can use google scholar to find a model that generates such networks, you do not need to invent them from scratch.

## Part 1: Data Generation

The five centralities that we will examine first are degree centrality, closeness centrality, betweeness centrality, load flow centrality, and reaching centrality. All of the data for all of the graphs will be loaded into centrality_data.csv. A copy of the data will be put into data.csv just in case centrality_data.csv gets over-written. All in all, the code takes about an hour to run on Tim's machine.

In [13]:
import pandas as pd
import numpy as np
import networkx as nx

In [14]:
def get_degrees(G):
    deg = nx.degree_centrality(G)
    deg_df = pd.Series(deg).to_frame()
    deg_df.columns = ['Degree']
    deg_df['degree_rank'] = deg_df['Degree'].rank(method = 'min', ascending = False)
    return deg_df

In [15]:
def get_closeness(G):
    deg = nx.degree_centrality(G)
    deg_df = pd.Series(deg).to_frame()
    deg_df.columns = ['Closeness']
    deg_df['closeness_rank'] = deg_df['Closeness'].rank(method = 'min', ascending = False)
    return deg_df

In [16]:
def get_betweeness(G):
    deg = nx.betweenness_centrality(G)
    deg_df = pd.Series(deg).to_frame()
    deg_df.columns = ['Betweeness']
    deg_df['betweeness_rank'] = deg_df['Betweeness'].rank(method = 'min', ascending = False)
    return deg_df

In [17]:
def get_current_flow(G):
    deg = nx.load_centrality(G)
    deg_df = pd.Series(deg).to_frame()
    deg_df.columns = ['Load']
    deg_df['load_rank'] = deg_df['Load'].rank(method = 'min', ascending = False)
    return deg_df

In [18]:
def get_global_reaching(G):
    deg = nx.load_centrality(G)
    deg_df = pd.Series(deg).to_frame()
    deg_df.columns = ['Reaching']
    deg_df['reach_rank'] = deg_df['Reaching'].rank(method = 'min', ascending = False)
    return deg_df

Networks and Parameters:

|Network Types | Parameter(s) |
|--------------|--------------|
|scale-free| {size : How many nodes to generate} |
|small-world | {size : How many nodes to generate, k : k nearest neighbor in a ring, p : probability of rewiring each edge}|
|random | {size : How many nodes to generate, n : number of edges}|
|scale-free & small-world |{size : How many nodes to generate, m : number of random edges for each node, p : probability of rewiring each edge} |

In [20]:
# DO NOT RUN THIS CELL. IT WILL OVERWRITE DATA AND TAKE AN HOUR TO RUN.
with open('centrality_data.csv', 'w') as wr:
    # Write the column headers to csv
    wr.write('networkType, networkSize, instanceNumber, nodeNumber, degreeRank, closenessRank, betweennessRank, loadRank, reachRank\n')
    networkTypes = ['scale-free', 'small-world', 'random', 'small-world/scale-free'] # networks to add
    sizes = [100, 200, 400, 800] # Sizes to add
    G = nx.scale_free_graph(100) # Initialization of the graph so the scope of the variable is outside the loops
    
    for netType in networkTypes:
        for size in sizes:
            for instNum in range(1, 26):
                if netType == 'scale-free':
                    G = nx.scale_free_graph(size)
                elif netType == 'small-world':
                    G = nx.watts_strogatz_graph(size, 3, 0.5)
                elif netType == 'small-world/scale-free':
                    G = nx.powerlaw_cluster_graph(size, 3, 0.5)
                else:
                    G = nx.gnm_random_graph(size, size * 4)
                degree_list = get_degrees(G)
                closeness_list = get_closeness(G)
                betweeness_list = get_betweeness(G)
                load_list = get_current_flow(G)
                reach_list = get_global_reaching(G)
                for node in G.nodes():
                    wr.write(netType + ', ' + str(size)+ ', ' + str(instNum) + ', ')
                    wr.write(str(node) + ', ' + str(degree_list['degree_rank'][node]) + ', ')
                    wr.write(str(closeness_list['closeness_rank'][node]) + ', ')
                    wr.write(str(betweeness_list['betweeness_rank'][node]) + ', ')
                    wr.write(str(load_list['load_rank'][node]) + ', ')
                    wr.write(str(reach_list['reach_rank'][node]) + '\n')
                    

In milestone 1, we failed to correctly generate a network that had both the properties of a scale-free & small-world network. After doing research, we came across this research paper: https://www.uvm.edu/pdodds/files/papers/others/2002/holme2002a.pdf . These researchers were able to generate a network that had both of the network properties. Networkx has a function implemented that will generate a random graph with this algorithm and it is demonstrated below.

In [11]:
G = nx.powerlaw_cluster_graph(100, 3, 0.5)

In [12]:
import matplotlib.pyplot as plt
%matplotlib inline
pos = nx.spring_layout(G)
nx.draw_networkx(G, pos, with_labels=False, node_size=50)